# Wash trades

__DEFINITION__

2 trades filled by a market maker that share a transaction hash where the maker/taker tokens of the first order are the taker/maker tokens of the second

## Prerequisites
1. Install `psycopg2` 
    1. on macos, also need `brew install postgresql` to install
    2. `pip3 install psycopg2`
2. Install `pandas`
3. Have access to a read only user to the database
    1. DM perry for database credentials 

### constants

In [1]:
API_ROOT = "https://api.rook.fi/api/v1/"
MM_ENDPOINT = f"{API_ROOT}coordinator/marketMakers"
AUCTIONS_ENDPOINT = f"{API_ROOT}coordinator/auctions"

# connection constants
PG_USER = "readonly"
PG_PWD = ""
PG_HOST = ""
PG_DB = "staging"

### imports

In [2]:
import math
import psycopg2
import requests
import time

import pandas as pd

from collections import defaultdict

### db connection

In [3]:
conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DB,
    user=PG_USER,
    password=PG_PWD)

In [4]:
cursor = conn.cursor()

### functions

In [5]:
def get_mms():
    response = requests.get(MM_ENDPOINT).json()
    return {mm["name"]: {"addresses": mm["makerAddresses"]} for mm in response}

In [6]:
def get_order_fills(mm_addresses, cursor):
    ofs = []
    cursor.execute("SELECT * FROM public.orderfill")
    for order_fill in cursor:
        txHash = order_fill[0]
        orderHash = order_fill[1]
        maker = order_fill[2]
        makerToken = order_fill[4]
        takerToken = order_fill[5]
        timestamp = order_fill[-1]
        
        ofs.append(
            {
                "txHash": txHash,
                "orderHash": orderHash,
                "makerToken": makerToken,
                "takerToken": takerToken,
                "maker": maker,
                "timestamp": timestamp,
            }
        )
            

    return ofs


In [7]:
mms = get_mms()
reverse_mms = {
    address: name for name, mm_obj in mms.items() for address in mm_obj["addresses"]
}
get_maker_name = lambda address: reverse_mms.get(address, None)

In [8]:
def get_wash_trades(order_fills):
    txHash_matched_orderFills = defaultdict(list)
    for order_fill in order_fills:
        txHash_matched_orderFills[order_fill["txHash"]].append(order_fill)

    # wash trades have to have at least 2 orderFills
    potential_wash_trades = {
        txHash: orderFills
        for txHash, orderFills in txHash_matched_orderFills.items()
        if len(orderFills) > 1
    }
    
    wash_trades = []
    for txHash, orderFills in potential_wash_trades.items():
        
        for i in range(len(orderFills) - 1):
            maker1Name = get_maker_name(orderFills[i]["maker"])
            for j in range(i + 1, len(orderFills)):
                maker2Name = get_maker_name(orderFills[j]["maker"])
                
                # constraints to be a wash trade: 
                # 1. makerToken 1 == takerToken 2
                # 2. takerToken 1 == makerToken 2
                # 3. makerName 1 == makerName 2 OR either maker name is known and the other is None
                makerTokenConstraint = orderFills[i]["makerToken"] == orderFills[j]["takerToken"]
                takerTokenConstraint = orderFills[i]["takerToken"] == orderFills[j]["makerToken"]
                makerConstraint = (maker1Name is not None) and (maker2Name is not None) and (maker1Name == maker2Name)
                
                if (
                    makerTokenConstraint and takerTokenConstraint and makerConstraint
                ):
                    makerName = maker1Name if maker1Name else maker2Name
                    wash_trades.append(
                        (
                            orderFills[i]["txHash"],
                            orderFills[i]["orderHash"],
                            orderFills[j]["orderHash"],
                            orderFills[i]["maker"],
                            orderFills[i]["timestamp"]
                        )
                    )
    return wash_trades

In [9]:
def get_bid_amounts(order_hashes):
    bids = {order_hash: 0 for order_hash in order_hashes}
    
    for i in range(math.ceil(len(order_hashes) / 30)):
        startIdx = i * 30
        endIdx = (i + 1) * 30
    
        request_params = {"orderHashes": order_hashes[startIdx:endIdx]}
        response = requests.get(AUCTIONS_ENDPOINT, request_params).json()
        
        for auction in response:
            winning_bids = [
                bid["rook_etherUnits"]
                for bid in auction["bidList"]
                if bid["outcome"] and bid["outcome"]["outcomeValue"] > 0
            ]
            if not len(winning_bids):
                continue
            if len(winning_bids) > 1:
                print(f"{auction['orderHash']} has {len(winning_bids)} winning bids")
            bids[auction["orderHash"]] = winning_bids[0]
    return bids

### Execute 

In [10]:
addresses = [address for _, mm_obj in mms.items() for address in mm_obj["addresses"]]

In [11]:
order_fills = get_order_fills(addresses, cursor)

In [12]:
wash_trades = get_wash_trades(order_fills)
print(f"Found {len(wash_trades)} potential wash trades")

Found 123 potential wash trades


In [13]:
orderHashes = list(
        set(
            [wash_trade[1] for wash_trade in wash_trades]
            + [wash_trade[2] for wash_trade in wash_trades]
        )
    )

In [14]:
bid_amounts = get_bid_amounts(orderHashes)

In [15]:
data = []
for wash_trade in wash_trades:
    (txHash, orderHash1, orderHash2, makerAddress, timestamp) = wash_trade
    maker_name = reverse_mms[makerAddress] if makerAddress in reverse_mms else "N/A"
    bidAmount = bid_amounts[orderHash1]
    bidAmount2 = bid_amounts[orderHash2]
    if bidAmount and not bidAmount2:
        bidAmount2 = bidAmount
    elif bidAmount2 and not bidAmount:
        bidAmount = bidAmount2
    if bidAmount != bidAmount2:
        print(f"bad data for\ntxHash={txHash}, orderHash1={orderHash1}, orderHash2={orderHash2}")
        continue
        
    data.append((maker_name, makerAddress, orderHash1, orderHash2, txHash, bidAmount, timestamp))
washTrade_df = pd.DataFrame(data=data, columns=["MarketMaker", "MakerAddress", "OrderHash1", "OrderHash2", "txHash", "bidAmount", "timestamp"])

    

## All potential wash trades for all time

In [16]:
print(washTrade_df.to_string())

    MarketMaker                                MakerAddress                                                          OrderHash1                                                          OrderHash2                                                              txHash  bidAmount   timestamp
0    VolleyFire  0x96aea3a04627f96a038b348b4d34ac24df08820a  0xd4f60b3437961b91234d28487964193d0d65a7de4e739ffbba82aae472888ed4  0x06dcb5ff208d8052dcb0fca5e4571be9220adaf504eb63bb3134b92569448c9a  0x5939a6d502652e5b3dd552e31ee5ef8487ba7fcc8ce0fa023ab3fc324f9261c2   0.000000  1632167083
1    VolleyFire  0x7b1886e49ab5433bb46f7258548092dc8cdca28b  0xcd6c470b138d8c772f4e5ba8e397572ac57c05a0115cc4c7a62fa0311a5dbfe9  0x5c47aad9771b01e61c7a3be6691bd3f4825c2fdb831a9a9fc604192d75dbf40c  0x066edb5f6aa88399cd951e4d3d3691b376e816ecb4402debc13e48037e564a7c   3.788693  1650255652
2    VolleyFire  0x4a45afd5a9691407b2b8e6ed8052a511ee7f01e9  0x0ee9f19830ce27387fb526b0c3d5785588a9eb57683f0ad617f6aae6d1500f23  0x2768d5e2

## Wash trades since pilot program start

In [17]:
# num_days = 1
# start_time = time.time() - (86400 * num_days)
start_time = 1650153600 # first pilot program epoch date

In [18]:
pp_wash_trades = washTrade_df.loc[washTrade_df['timestamp'] >= start_time]

In [19]:
print(pp_wash_trades.to_string())

    MarketMaker                                MakerAddress                                                          OrderHash1                                                          OrderHash2                                                              txHash  bidAmount   timestamp
1    VolleyFire  0x7b1886e49ab5433bb46f7258548092dc8cdca28b  0xcd6c470b138d8c772f4e5ba8e397572ac57c05a0115cc4c7a62fa0311a5dbfe9  0x5c47aad9771b01e61c7a3be6691bd3f4825c2fdb831a9a9fc604192d75dbf40c  0x066edb5f6aa88399cd951e4d3d3691b376e816ecb4402debc13e48037e564a7c   3.788693  1650255652
2    VolleyFire  0x4a45afd5a9691407b2b8e6ed8052a511ee7f01e9  0x0ee9f19830ce27387fb526b0c3d5785588a9eb57683f0ad617f6aae6d1500f23  0x2768d5e2ffd0353abd52c20db73d2d964f168d46ad2839fea8de6fcd3440584a  0x7d1f3a9da8c9696937fecc2a5b6bc2e86038f0c8c060d514e91bf9eadce64f6c   0.011904  1650362587
3    VolleyFire  0xcf9ebecaa4b1581b5566baacfb6d9933f9849032  0xf6f2344d3587b1fc950c95901f03e46d365d1de67cced0c8c6c328f25b5d372a  0x10b1e3f4

In [20]:
print(pp_wash_trades.shape[0])

122


In [21]:
pp_wash_trades.to_csv("~/Desktop/pilotProgramWashTrades.csv")